# Conversión de coordenadas

El proceso de conversión de coordenadas puede resultar conflictivo, ya que las imágenes en png pierden la georeferencia, por lo que será necesario buscar una forma de recuperarla.

Para ello, debemos dividir el problema en dos partes:

**1. Mantener el identificador único de la imagen**
Cada imagen en formato PNG que los ciudadanos marquen debe tener un nombre único que también incluya información sobre su ubicación en el ortomosaico georreferenciado. Este nombre podría incluir un identificador del "tile" o fragmento de la imagen (por ejemplo, una celda de un grid), así como un código o nombre que permita hacer la correspondencia con el ortomosaico. Por ejemplo, el nombre de la imagen podría ser algo como tile_001_001.png, donde 001_001 hace referencia a un fragmento o sección dentro del ortomosaico.


In [1]:

import os
import csv
#import rasterio
import pandas as pd



# Path to your TIFF files
input_path = os.path.join('G:', '.shortcut-targets-by-id', '1pYgV5EIk4-LapLNhlCwpQaDAzuqNffXG', 'doctorado_albert', 'Pinguiton', 'ortho_enero_500')
tiff_files = [file for file in os.listdir(input_path) if file.endswith(('.tif', '.TIF'))]

# CSV output file
csv_file = "../data/ortho_coordinates.csv"
os.makedirs(os.path.dirname(csv_file), exist_ok=True)

# TODO: definir la función que va a recortar con rasterio en nuevos tiffs pero manteniendo las georeferencias.

def cut_tile(tiff_path:str, output_path:str) -> None:
    return None


**2. Definir una relación entre las imágenes y el ortomosaico**

Cada imagen individual (como tile_001_001.png) debe tener una relación precisa con su posición dentro del ortomosaico completo. Esto quiere decir que al recortar cada tile en subrecortes -también en tiff- podríamos crear un csv donde se guarde el nombre de la imagen junto con sus esquinas superior izquierda e inferior dercha. De talf orma, que el csv podría tener la siguiente estructura:

```csv
    file,upper_left,bottom_right
    imagen-43-58.tif,"(695041.744254731, 3086365.36109968)","(695052.174854731, 3086354.93049968)"
    imagen-119-120.tif,"(695834.469854731, 3085718.6638996797)","(695844.900454731, 3085708.2332996796)"
    imagen-53-99.tif,"(695146.050254731, 3085937.70649968)","(695156.4808547309, 3085927.27589968)"
```

o también podría ser:

file, upper_x, upper_y, bottom_x, bottom_y

In [2]:

# Function to get bounds of a TIFF file
def get_image_bounds(tiff_path:str) -> tuple:
    """
    Devuelve la esquina superior izquierda y la esquina inferior derecha de la imagen seleccionada.
    La función transform*(0,0) se encarga de devolver el sistema georeferneciasa del archivo (NO en píxeles).

    Args:
    - tiff_path: str, ruta completa del archivo TIFF.
    """
    with rasterio.open(tiff_path) as src:
        transform = src.transform
        width, height = src.width, src.height

        # Upper-left corner
        ul_x, ul_y = transform * (0, 0)
        # Bottom-right corner
        br_x, br_y = transform * (width, height)
        return ul_x, ul_y, br_x, br_y



# Writing to CSV
with open(csv_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["file", "x_top", "y_top", "x_bot", "y_bot"])

    # Process and write all TIFF files
    count = 0
    for file_name in tiff_files:
        count += 1
        if count > 10:
            break
    
        tiff_path = os.path.join(input_path, file_name)  
        x_top, y_top, x_bot, y_bot = get_image_bounds(tiff_path)

        # Remove file extension
        file_name = file_name.split('.')[0]
        writer.writerow([file_name, x_top, y_top , x_bot, y_bot ])

print(f"CSV file '{csv_file}' has been created.")

CSV file '../data/ortho_coordinates.csv' has been created.



**3. Recuperar la georreferenciación cuando se marca un píxel**

Cuando los ciudadanos marqune una posición, estarán marcando un pixel. Pero en realidad para nosotros esta fase es indiferente, ya que simplemente tenemos que revertir la posición de dicho pixel a su georeferencia sabiendo que la posición máxima y mínima de referencia ya están guardadas en el archivo csv. 

Para ello tendrmeos dos documentos:

- Un csv con las coordenadas de las esquinas de cada imagen + nombre (la superior izquierda e inferior derecha).
- Un json sacado de la base de datos con las coordenadas de cada pixel marcado + nombre imagen.

In [ ]:
# Cargar el archivo CSV con las coordenadas de las esquinas

pixel_coordinates_json = "../data/pixel_coordinates.json"
df_pixel_coordinates = pd.read_json(pixel_coordinates_json)
df_img_box_coords = pd.read_csv(csv_file)

# Función para convertir píxeles a coordenadas geográficas
def pixel_to_coordinates(
    tile_name:str, 
    pixel_x:int, 
    pixel_y:int, 
    csv_corners_data: pd.DataFrame,
    path_tiff_files: str
):
    """
    Esta función está pensada para llamarse dentro de un bucle for donde se itere 
    sobre el JSON con todos los tiles con los píxeles marcados.

    Convierte las coordenadas del píxel (pixel_x, pixel_y) de una imagen (tile) a coordenadas geográficas
    usando la información de los límites geográficos (upper_left, bottom_right) que se encuentra en el CSV.
    
    Args:
    - tile_name (str): Nombre de la imagen/fragmento del ortomosaico.
    - pixel_x (int): Posición X del píxel marcado en la imagen.
    - pixel_y (int): Posición Y del píxel marcado en la imagen.
    - csv_corners_data (pd.DataFrame): DataFrame con las coordenadas de las esquinas de las imágenes.
    - path_img_files: path a donde se encuentran las imágenes para medir su ancho y alto.
    Returns:
    - tuple: Coordenadas geográficas (x, y).
    """

    # Obtenemos el alto y ancho de la imagen
    tiff_path = os.path.join(path_tiff_files, tile_name, '.tif')
    with rasterio.open(tiff_path) as src:
        width, height = src.width, src.height
    print(f"Ancho de la imagen: {width} píxeles")
    print(f"Alto de la imagen: {height} píxeles")

    # Buscar el tile correspondiente en el CSV y extraer sus límites
    tile_row = csv_corners_data[csv_corners_data['file'] == tile_name].iloc[0]
    upper_left = eval(tile_row['upper_left'])
    bottom_right = eval(tile_row['bottom_right'])
    

    # Calcular las proporciones de las coordenadas geográficas
    x_ratio = pixel_x / width
    y_ratio = pixel_y / height
    
    #! Calcular la coordenada geográfica correspondiente: ASEGURAR QUE FUNCIONA EN POLO SUR
    geo_x = upper_left[0] + x_ratio * (bottom_right[0] - upper_left[0])
    geo_y = upper_left[1] + y_ratio * (bottom_right[1] - upper_left[1])
    
    return geo_x, geo_y

path_tiff_files = os.path.join('G:', '.shortcut-targets-by-id', '1pYgV5EIk4-LapLNhlCwpQaDAzuqNffXG', 'doctorado_albert', 'Pinguiton', 'ortho_enero_500')

# Ejemplo de uso:
for _, row in df_pixel_coordinates.iterrows():
    tile_name = row['file']
    pixel_x = row['pixel_x']
    pixel_y = row['pixel_y']
  
    geo_x, geo_y = pixel_to_coordinates(tile_name, pixel_x, pixel_y, df_img_box_coords, path_tiff_files)
    print(f"Coordenadas geográficas del píxel: ({geo_x}, {geo_y})")
